In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [2]:
csl_df = pd.read_csv('../../data/csl/Sites/CSL_he_PI_s45.csv', index_col=0)
X = csl_df.drop(['high_EBL','Sitenum'], axis=1, inplace=False)
y = csl_df['high_EBL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=7)
X_train                                                    

,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
45-01244,0,0,9,3,2,99.0,87,100.0,1,131.0,...,0,0,0,0,0,0,28,24,1,1
45-11378,0,0,8,3,3,10.0,79,60.0,1,125.0,...,0,0,0,0,1,0,38,24,1,1
45-14054,0,0,1,3,1,10.0,68,30.0,1,129.0,...,0,0,0,0,0,0,25,23,1,1
45-01134,0,0,8,8,8,0.0,84,0.0,77,146.0,...,0,0,0,0,1,0,26,24,1,1
45-12058,0,0,8,8,8,99.0,67,0.0,77,117.0,...,0,0,0,0,1,0,34,24,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45-14585,0,0,9,1,3,2.0,62,100.0,1,126.0,...,0,0,0,0,0,0,23,24,1,1
45-14580,0,0,10,2,3,2.0,75,100.0,1,142.0,...,0,0,0,0,0,0,30,37,1,1
45-12005,0,0,5,3,2,10.0,81,70.0,1,120.0,...,0,0,0,0,0,0,29,24,1,1


In [3]:
X.shape

(11879, 195)

<h1 align='center'>Exploratory Data Analysis</h1>

<h1 align='center'>Modelling</h1>

In [8]:
print(X_train.shape)
print(X.shape)
print(y.shape)

(8315, 195)
(11879, 195)
(11879,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [9]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [10]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(910, 195)
(910,)
[455 455]


<h1 align='center'>Basic Gradient Boosting</h1>

In [11]:
%%time
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
gb_rus.fit(X_rus, y_rus)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[2516  853]
 [  15  180]]
              precision    recall  f1-score   support

           0       0.99      0.75      0.85      3369
           1       0.17      0.92      0.29       195

    accuracy                           0.76      3564
   macro avg       0.58      0.83      0.57      3564
weighted avg       0.95      0.76      0.82      3564

ROC AUC prob1: 0.8699423857037393
MCC: 0.33579368284763844
F2: 0.4964147821290679
PR_AUC: 0.2388490561225601
CPU times: user 356 ms, sys: 2.89 ms, total

In [12]:
%%time
# Not undersampled
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier()  
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[2585  784]
 [  23  172]]
              precision    recall  f1-score   support

           0       0.99      0.77      0.86      3369
           1       0.18      0.88      0.30       195

    accuracy                           0.77      3564
   macro avg       0.59      0.82      0.58      3564
weighted avg       0.95      0.77      0.83      3564

ROC AUC prob1: 0.8722865340852876
MCC: 0.3333174753313989
F2: 0.4953917050691244
PR_AUC: 0.25901781882089486
CPU times: user 2.43 s, sys: 4.73 ms, total

<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [13]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)

#        weights = class_weight.compute_sample_weight('balanced', y=y_train)
#        model.fit(X_train, y_train, sample_weight=weights)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-10 14:36:34,126] A new study created in memory with name: no-name-ecf0f3f9-6128-463b-ab5b-0f45b2b8b571
[I 2021-12-10 14:36:35,117] Trial 0 finished with value: 0.33710350552060636 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.33710350552060636.


MCC: [0.33177477443291925, 0.3377050812811679, 0.3572127099169114, 0.32172145645142697]
ROC: [0.8650021103724806, 0.8657497360892248, 0.883953569188335, 0.8618159187568997]
PR_AUC: [0.22835811001551834, 0.23224578396845674, 0.27644156659960956, 0.273563877740759]


[I 2021-12-10 14:36:37,445] Trial 1 finished with value: 0.3350170315799119 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 0 with value: 0.33710350552060636.


MCC: [0.34486724625444226, 0.3345071846055076, 0.3462679340494135, 0.3144257614102843]
ROC: [0.8564023425134537, 0.8550359421341477, 0.8781420681425272, 0.8437130278360536]
PR_AUC: [0.20319419618828943, 0.1899569645436858, 0.28221838077149436, 0.1969877995037051]


[I 2021-12-10 14:36:39,617] Trial 2 finished with value: 0.3291949994384282 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 0 with value: 0.33710350552060636.


...

MCC: [0.32230392265835117, 0.334774684036634, 0.3486420783218614, 0.3352162957718959]
ROC: [0.8613815025852063, 0.853235010632927, 0.8894590867266539, 0.84952961511565]
PR_AUC: [0.22293376380518018, 0.1846623138693883, 0.282316130314597, 0.2063613221264443]


[I 2021-12-10 14:37:30,211] Trial 23 finished with value: 0.3391669506901568 and parameters: {'n_estimators': 33, 'subsample': 0.8504413878499427, 'max_depth': 14, 'max_features': 19, 'learning_rate': 0.10790897273562051, 'min_samples_leaf': 32}. Best is trial 21 with value: 0.3438941189228.


MCC: [0.3366922183467397, 0.3365578564637117, 0.34555750217237924, 0.3378602257777966]
ROC: [0.8584225845028314, 0.8577701233332096, 0.8825591586327782, 0.8637731069152514]
PR_AUC: [0.22556252289009704, 0.21368525784819106, 0.2749260070532586, 0.251691211098758]


[I 2021-12-10 14:37:33,514] Trial 24 finished with value: 0.32744371968335356 and parameters: {'n_estimators': 101, 'subsample': 0.8799731267520511, 'max_depth': 17, 'max_features': 16, 'learning_rate': 0.09533328289091661, 'min_samples_leaf': 21}. Best is trial 21 with value: 0.3438941189228.


MCC: [0.3230233920535266, 0.33324716138739163, 0.3375649963243928, 0.3159393289681031]
ROC: [0.8561693222187049, 0.8608845983201505, 0.8860058442849931, 0.8618796923036325]
PR_AUC: [0.2151144361678626, 0.2118860697301815, 0.26785356805155225, 0.273090292465858]
CPU times: user 59.4 s, sys: 110 ms, total: 59.6 s
Wall time: 59.4 s


In [14]:
best = study.best_trial
best

FrozenTrial(number=21, values=[0.3438941189228], datetime_start=datetime.datetime(2021, 12, 10, 14, 37, 26, 145911), datetime_complete=datetime.datetime(2021, 12, 10, 14, 37, 27, 822275), params={'n_estimators': 58, 'subsample': 0.8498502075713545, 'max_depth': 15, 'max_features': 19, 'learning_rate': 0.09378400776091556, 'min_samples_leaf': 31}, distributions={'n_estimators': IntUniformDistribution(high=200, low=10, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=20, low=3, step=1), 'max_features': IntUniformDistribution(high=20, low=10, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.08), 'min_samples_leaf': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=21, state=TrialState.COMPLETE, value=None)

In [16]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb_weight = GradientBoostingClassifier(**gb_params)
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[2833  536]
 [  63  132]]
              precision    recall  f1-score   support

           0       0.98      0.84      0.90      3369
           1       0.20      0.68      0.31       195

    accuracy                           0.83      3564
   macro avg       0.59      0.76      0.61      3564
weighted avg       0.94      0.83      0.87      3564

ROC AUC prob1: 0.8727173094047538
MCC: 0.30176138809434633
F2: 0.45580110497237575
PR_AUC: 0.23143854093057512
CPU times: user 889 ms, sys: 980 µs, total: 890 ms
Wall time: 887 ms


In [15]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[2551  818]
 [  16  179]]
              precision    recall  f1-score   support

           0       0.99      0.76      0.86      3369
           1       0.18      0.92      0.30       195

    accuracy                           0.77      3564
   macro avg       0.59      0.84      0.58      3564
weighted avg       0.95      0.77      0.83      3564

ROC AUC prob1: 0.8702726975211391
MCC: 0.3420624851489895
F2: 0.5036578503095104
PR_AUC: 0.21139062898627556
CPU times: user 138 ms, sys: 3.02 ms, total: 141 ms
Wall time: 139 ms


In [17]:
coeffs_wgt = pd.Series(data=gb_weight.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs_wgt[0:20]

Presentdel     0.285902
Delmode        0.063847
Lac_None       0.058451
CS_FTP         0.055910
AdmSBP         0.038917
AdmDBP         0.034367
TrialLabor     0.033963
new_age        0.033229
Admefface      0.020668
Delfetalpos    0.019940
AdmBishop      0.019468
CS_NRFHT       0.019427
Dilat_lst      0.017265
momrace_new    0.016374
prelaborCD     0.016193
BESTGA         0.016166
Admcontract    0.015027
Lac_Min        0.012726
Admcervpos     0.011324
Analgesia      0.010709
dtype: float64

In [26]:
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Presentdel        0.248502
Delfetalpos       0.142930
CS_FTP            0.072793
Lac_Min           0.057240
Delmode           0.055049
Dilat_lst         0.043160
AdmSBP            0.035636
Admefface         0.025463
AdmDBP            0.025085
Admconsistency    0.024490
BESTGA            0.019899
new_age           0.018413
TrialLabor        0.016620
Admcontract       0.016097
Lac_None          0.015849
spontlabor        0.013282
AdmBishop         0.011382
ROMmeth           0.010288
new_BMI           0.009608
prelaborCD        0.009599
dtype: float64

In [1]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs_wgt.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs_wgt.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'plt' is not defined

In [19]:
# No Sitenum - 5
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Presentdel        0.248502
Delfetalpos       0.142930
CS_FTP            0.072793
Lac_Min           0.057240
Delmode           0.055049
Dilat_lst         0.043160
AdmSBP            0.035636
Admefface         0.025463
AdmDBP            0.025085
Admconsistency    0.024490
BESTGA            0.019899
new_age           0.018413
TrialLabor        0.016620
Admcontract       0.016097
Lac_None          0.015849
spontlabor        0.013282
AdmBishop         0.011382
ROMmeth           0.010288
new_BMI           0.009608
prelaborCD        0.009599
dtype: float64

In [21]:
gb_params # 5 runs

{'n_estimators': 58,
 'subsample': 0.8498502075713545,
 'max_depth': 15,
 'max_features': 19,
 'learning_rate': 0.09378400776091556,
 'min_samples_leaf': 31,
 'random_state': 42}

In [3]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [ ]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

In [24]:
np.mean([0.3149689279176914, 0.31449457458602254, 0.31066798538596907, 0.3219519143488398])

0.3155208505596307

<h1 align='center'>Thanks and make sure to learn!</h1>